In [49]:
import pandas as pd
import re
import numpy as np
import random
from pdb import set_trace as bp

# reading csv file
df_orig = pd.read_csv(r"I:\Problem 1-dataset.csv")
df = np.array(df_orig)
print(df.shape)

(20505, 4)


In [50]:
cleandf = []
labels = []

for i in range(len(df)):

    if (df[i, 1] != "label"):
        line = df[i, :].tolist()
        word_list = re.split(':|,|#|/| ', line[0])
        for word in word_list:
          if word != '' and word != '-':
            line.append(word)
        
        if line[2] is None:
            line[2] = '-1'
        
        if line[3] is None:
            line[3] = '-1'
            
        cleandf.append(line)
        #print(line)
        #print(cleandf)
        
        if df[i, 1] not in labels:
            labels.append(df[i, 1])

    
labels.sort()






   






In [51]:
x = list(range(len(cleandf)))
random.shuffle(x)

trainlist = cleandf[0:int(len(x) * 0.7) - 1]
testlist = cleandf[(len(cleandf) - len(trainlist) - 1):]


# build dic from train
dict = []
wfreq = []
for i in range(len(cleandf)):
    word_list = cleandf[i][4:]
    #print(word_list)
    for wid in word_list:
        if (wid in dict):
            wfreq[dict.index(wid)] += 1
        else:
            dict.append(wid);
            wfreq.append(1)

class_items = []
class_prob = []


for j, label in enumerate(labels):
    templist = []
    for item in cleandf:
        if (item[1] == label):
            templist.append(item)
    class_items.append(templist)
            
#print(class_items[0][0][1:3])

class_word_prob = []
for i, label in enumerate(labels):
    temp_count = np.zeros(len(dict))
    for rd in class_items[i]:
        word_list = rd[4:]
        for wid in word_list:
            if wid in dict:
                temp_count[dict.index(wid)] += 1
                
    class_word_prob.append(temp_count)
    class_prob.append(len(class_items[i]))
   
    class_word_prob[i][:] = np.divide(class_word_prob[i][:], wfreq)
    class_prob[i] /= len(cleandf)
    
print(class_prob)




[0.1975609756097561, 0.17658536585365853, 0.21365853658536585, 0.2097560975609756, 0.20243902439024392]


In [52]:
confusion_table = np.zeros([len(labels), len(labels)]);

predict_label = 0
predict_accuracy = 0
for i in range(len(testlist)):
    word_list = testlist[i][4:]
    predict_label = 0;
    max = 0;
    for j, label in enumerate(labels):
        testsample_prob = 0

        for wid in word_list:
            ind = dict.index(wid)
            testsample_prob += class_word_prob[j][ind]

        testsample_prob = testsample_prob * class_prob[j]

        if (testsample_prob > max):
            max = testsample_prob
            predict_label = j

    if (labels[predict_label] == testlist[i][1]):
        predict_accuracy += 1
        confusion_table[predict_label,predict_label] = confusion_table[predict_label,predict_label] + 1
    else:
        confusion_table[labels.index(testlist[i][1]), predict_label] += 1
   
predict_accuracy /= len(testlist)

print(predict_accuracy)
print(confusion_table)
    




0.9958185239389504
[[2811.    0.    0.    0.    0.]
 [   0. 2506.   15.   15.   15.]
 [   0.    0. 3039.    0.    0.]
 [  15.    0.    0. 3005.    0.]
 [   0.    0.    0.    0. 2928.]]
